## LangGraph Open Deep Research - Supervisor-Researcher Architecture

In this notebook, we'll explore the **supervisor-researcher delegation architecture** for conducting deep research with LangGraph.

You can visit this repository to see the original application: [Open Deep Research](https://github.com/langchain-ai/open_deep_research)

Let's jump in!

## What We're Building

This implementation uses a **hierarchical delegation pattern** where:

1. **User Clarification** - Optionally asks clarifying questions to understand the research scope
2. **Research Brief Generation** - Transforms user messages into a structured research brief
3. **Supervisor** - A lead researcher that analyzes the brief and delegates research tasks
4. **Parallel Researchers** - Multiple sub-agents that conduct focused research simultaneously
5. **Research Compression** - Each researcher synthesizes their findings
6. **Final Report** - All findings are combined into a comprehensive report

![Architecture Diagram](https://i.imgur.com/Q8HEZn0.png)

This differs from a section-based approach by allowing dynamic task decomposition based on the research question, rather than predefined sections.

---

# 🤝 Breakout Room #1
## Deep Research Foundations

In this breakout room, we'll understand the architecture and components of the Open Deep Research system.

## Task 1: Dependencies

You'll need API keys for Anthropic (for the LLM) and Tavily (for web search). We'll configure the system to use Anthropic's Claude Sonnet 4 exclusively.

In [7]:
import os
import getpass
# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key: ")
os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

## Task 2: State Definitions

The state structure is hierarchical with three levels:

### Agent State (Top Level)
Contains the overall conversation messages, research brief, accumulated notes, and final report.

### Supervisor State (Middle Level)
Manages the research supervisor's messages, research iterations, and coordinating parallel researchers.

### Researcher State (Bottom Level)
Each individual researcher has their own message history, tool call iterations, and research findings.

We also have structured outputs for tool calling:
- **ConductResearch** - Tool for supervisor to delegate research to a sub-agent
- **ResearchComplete** - Tool to signal research phase is done
- **ClarifyWithUser** - Structured output for asking clarifying questions
- **ResearchQuestion** - Structured output for the research brief

Let's import these from our library: [`open_deep_library/state.py`](open_deep_library/state.py)

In [8]:
# Import state definitions from the library
from open_deep_library.state import (
    # Main workflow states
    AgentState,           # Lines 65-72: Top-level agent state with messages, research_brief, notes, final_report
    AgentInputState,      # Lines 62-63: Input state is just messages
    
    # Supervisor states
    SupervisorState,      # Lines 74-81: Supervisor manages research delegation and iterations
    
    # Researcher states
    ResearcherState,      # Lines 83-90: Individual researcher with messages and tool iterations
    ResearcherOutputState, # Lines 92-96: Output from researcher (compressed research + raw notes)
    
    # Structured outputs for tool calling
    ConductResearch,      # Lines 15-19: Tool for delegating research to sub-agents
    ResearchComplete,     # Lines 21-22: Tool to signal research completion
    ClarifyWithUser,      # Lines 30-41: Structured output for user clarification
    ResearchQuestion,     # Lines 43-48: Structured output for research brief
)

## Task 3: Utility Functions and Tools

The system uses several key utilities:

### Search Tools
- **tavily_search** - Async web search with automatic summarization to stay within token limits
- Supports Anthropic native web search and Tavily API

### Reflection Tools
- **think_tool** - Allows researchers to reflect on their progress and plan next steps (ReAct pattern)

### Helper Utilities
- **get_all_tools** - Assembles the complete toolkit (search + MCP + reflection)
- **get_today_str** - Provides current date context for research
- Token limit handling utilities for graceful degradation

These are defined in [`open_deep_library/utils.py`](open_deep_library/utils.py)

In [9]:
# Import utility functions and tools from the library
from open_deep_library.utils import (
    # Search tool - Lines 43-136: Tavily search with automatic summarization
    tavily_search,
    
    # Reflection tool - Lines 219-244: Strategic thinking tool for ReAct pattern
    think_tool,
    
    # Tool assembly - Lines 569-597: Get all configured tools
    get_all_tools,
    
    # Date utility - Lines 872-879: Get formatted current date
    get_today_str,
    
    # Supporting utilities for error handling
    get_api_key_for_model,          # Lines 892-914: Get API keys from config or env
    is_token_limit_exceeded,         # Lines 665-701: Detect token limit errors
    get_model_token_limit,           # Lines 831-846: Look up model's token limit
    remove_up_to_last_ai_message,    # Lines 848-866: Truncate messages for retry
    anthropic_websearch_called,      # Lines 607-637: Detect Anthropic native search usage
    openai_websearch_called,         # Lines 639-658: Detect OpenAI native search usage
    get_notes_from_tool_calls,       # Lines 599-601: Extract notes from tool messages
)

## Task 4: Configuration System

The configuration system controls:

### Research Behavior
- **allow_clarification** - Whether to ask clarifying questions before research
- **max_concurrent_research_units** - How many parallel researchers can run (default: 5)
- **max_researcher_iterations** - How many times supervisor can delegate research (default: 6)
- **max_react_tool_calls** - Tool call limit per researcher (default: 10)

### Model Configuration
- **research_model** - Model for research and supervision (we'll use Anthropic)
- **compression_model** - Model for synthesizing findings
- **final_report_model** - Model for writing the final report
- **summarization_model** - Model for summarizing web search results

### Search Configuration
- **search_api** - Which search API to use (ANTHROPIC, TAVILY, or NONE)
- **max_content_length** - Character limit before summarization

Defined in [`open_deep_library/configuration.py`](open_deep_library/configuration.py)

In [10]:
# Import configuration from the library
from open_deep_library.configuration import (
    Configuration,    # Lines 38-247: Main configuration class with all settings
    SearchAPI,        # Lines 11-17: Enum for search API options (ANTHROPIC, TAVILY, NONE)
)

## Task 5: Prompt Templates

The system uses carefully engineered prompts for each phase:

### Phase 1: Clarification
**clarify_with_user_instructions** - Analyzes if the research scope is clear or needs clarification

### Phase 2: Research Brief
**transform_messages_into_research_topic_prompt** - Converts user messages into a detailed research brief

### Phase 3: Supervisor
**lead_researcher_prompt** - System prompt for the supervisor that manages delegation strategy

### Phase 4: Researcher
**research_system_prompt** - System prompt for individual researchers conducting focused research

### Phase 5: Compression
**compress_research_system_prompt** - Prompt for synthesizing research findings without losing information

### Phase 6: Final Report
**final_report_generation_prompt** - Comprehensive prompt for writing the final report

All prompts are defined in [`open_deep_library/prompts.py`](open_deep_library/prompts.py)

In [11]:
# Import prompt templates from the library
from open_deep_library.prompts import (
    clarify_with_user_instructions,                    # Lines 3-41: Ask clarifying questions
    transform_messages_into_research_topic_prompt,     # Lines 44-77: Generate research brief
    lead_researcher_prompt,                            # Lines 79-136: Supervisor system prompt
    research_system_prompt,                            # Lines 138-183: Researcher system prompt
    compress_research_system_prompt,                   # Lines 186-222: Research compression prompt
    final_report_generation_prompt,                    # Lines 228-308: Final report generation
)

## ❓ Question #1:

Explain the interrelationships between the three states (Agent, Supervisor, Researcher). Why don't we just make a single huge state?

##### Answer:
1. Parallel researchers
When the supervisor calls ConductResearch multiple times, several researchers run in parallel via asyncio.gather. Each invocation has its own ResearcherState. With a single shared state, all researchers would read and write the same fields (messages, notes, etc.) and overwrite each other’s work.
2. Different lifetimes
    a. AgentState: Exists for the whole run (clarify → brief → research → report).
    b. SupervisorState: Only during the research phase; it maps to AgentState fields when used as a subgraph.
    c. ResearcherState: Short-lived per topic; created for each ConductResearch call, runs to completion, returns ResearcherOutputState, then is discarded.
3. Clear input/output boundaries
The researcher subgraph is meant to be invoked many times with different topics. It expects ResearcherState and returns ResearcherOutputState. A monolithic state would blur these boundaries and make reuse and composition harder.
4. Context and token usage
Each level gets only what it needs:
    a. Researcher: one topic, not the full supervisor conversation.
    b. Supervisor: research brief and iteration info, not each researcher’s raw tool traces.
    c. Agent: messages, brief, notes, and final report.
That keeps prompts smaller and on-topic and helps control context size.
5. Separation of concerns
    a. Researcher: Run tools, gather info, compress findings.
    b. Supervisor: Decide what to research, delegate, iterate, decide when to stop.
    c. Agent: Clarify with user, plan research, orchestrate phases, produce final report.
Different roles need different state shapes and lifecycle management.

The three states exist because the system uses parallel, reusable subgraphs with different lifecycles and responsibilities. A single global state would break parallelism, blur roles, and complicate context management.

## ❓ Question #2:

What are the advantages and disadvantages of importing these components instead of including them in the notebook?

##### Answer:
Advantages of importing 
1. Notebook stays focused
The notebook can emphasize flow, architecture, and how pieces connect, without hundreds of lines of state, tools, and graph logic. Easier to read and follow the “story.”
2. Reuse and consistency
The same state, tools, and nodes are used everywhere. Fixes and improvements happen in one place (open_deep_library/) and apply to any notebook or script that imports them.
3. Easier to test
Code in modules can be unit-tested (e.g. state.py, utils.py, deep_researcher.py) with normal test runners. Notebooks are harder to test in the same way.
4. Version control and diffs
Changes to the library show up as clear, small diffs in .py files. Large notebooks with mixed code and output are noisier and harder to review.
5. Separation of concerns
“What the system does” (library) is separate from “how we run and explore it” (notebook). That separation scales better as the project grows.
6. IDE support
Go-to-definition, refactors, and type checking work better in .py files than in notebook cells.

Disadvantages of importing
1. Less visibility for learning
To see how state, tools, or the graph are implemented, you have to open state.py, utils.py, deep_researcher.py, etc. The notebook doesn’t show that code by default, so the “full picture” is split across files.
2. Context switching
Understanding one step (e.g. “what does supervisor do?”) requires jumping to another file instead of scrolling in the same document.
3. Setup and packaging
The package must be installed or on PYTHONPATH (e.g. pip install -e . or running from the repo root). If that isn’t done, imports fail and the cause is less obvious to someone new.
4. Debugging flow
Stepping through “notebook → library” with a debugger means crossing from notebook execution into library code. Some people find it easier to debug when everything is in one notebook.
5. Copy-paste and sharing
A single, self-contained notebook is easier to share (e.g. “run this one file”). With imports, you have to share the whole project or ensure the library is installable.
6. Discoverability
New learners might not realize they can (and should) open open_deep_library/ to see how things work; the notebook comments help by pointing to files like open_deep_library/state.py and line ranges.

The notebook is using imports and pointing you to the library (e.g. “see open_deep_library/state.py”) so you get:
A clean narrative in the notebook, and
The real implementation to read and modify in the library.
A good way to work is: read the notebook for the high-level flow, then open the referenced module when you want to see or change how something actually works. That combination gives you both clarity and depth.


## 🏗️ Activity #1: Explore the Prompts

Open `open_deep_library/prompts.py` and examine one of the prompt templates in detail.

**Requirements:**
1. Choose one prompt template (clarify, brief, supervisor, researcher, compression, or final report)
2. Explain what the prompt is designed to accomplish
3. Identify 2-3 key techniques used in the prompt (e.g., structured output, role definition, examples)
4. Suggest one improvement you might make to the prompt

**YOUR CODE HERE** - Write your analysis in a markdown cell below
1. Prompt: lead_researcher_prompt

2. Prompt is designed to achieve: 
    a. conduct research against the overall research question passed in by the user. 
    b. When completes the research on the topic call 'ResearchComplete' tool to indicate research done

3. Techniques: 

    a. Role definition: Setting up model identity with "you are a research supervisor" explicitly instructing that its not doing research, but deleigate research. It also sets up mindset of the model with constaints like "limited time and resources" 

    b. Structured sections: The prompt is split into clear blocks
        1. <Task> — What the agent must do (use ConductResearch, then call ResearchComplete when satisfied).
        2. <Available Tools> — What it can use (ConductResearch, ResearchComplete, think_tool) plus the CRITICAL rule about when to use think_tool.
        3. <Instructions> — Step-by-step workflow (read question → decide how to delegate → after each call, pause and assess).
        4. <Hard Limits> — Hard caps (e.g. max iterations, max parallel agents, “bias towards single agent”).
        5. <Show Your Thinking> — What to reflect on before/after ConductResearch.
        6. <Scaling Rules> — When to use 1 vs many sub-agents and important reminders.
    this structure separates goal, tools, procedure, constraints, and examples, which makes the prompt easier for the model to follow and for humans to edit.
       
    c. Examples/ scaling rules: 
    The Scaling Rules section gives concrete patterns:
    1. Single sub-agent:
            "Simple fact-finding, lists, and rankings can use a single sub-agent" with the example: "List the top 10 coffee shops in San Francisco → Use 1 sub-agent."
    2. Multiple sub-agents:
            "Comparisons presented in the user request can use a sub-agent for each element" with the example: "Compare OpenAI vs. Anthropic vs. DeepMind approaches to AI safety → Use 3 sub-agents" and "Delegate clear, distinct, non-overlapping subtopics."
     these act as few-shot examples that show when to use one agent vs several and how to split comparison-style questions so the model doesn’t over- or under-delegate.

4. Improvement: 
    a. Explicit "done" criteria: 
        Add 1–2 explicit ‘done’ conditions (e.g. coverage of each part of the question, or minimum number of distinct findings) so the model has a clear threshold for calling ResearchComplete. For example, adding this to prompt “You have at least one relevant finding for each main part of the user’s question.”

    b. Order of tools:
        Explicitly mention the order in which tools to be executed one-after-other. 
        “Always follow this loop: (1) think_tool to plan, (2) ConductResearch, (3) think_tool to assess, (4) if enough → ResearchComplete, else repeat from (1) until limits.”

    c. Handling Failure: Add a fallback rule: when the iteration or tool-call limit is reached and the question isn’t fully answered, still call ResearchComplete and require a short note in the final message explaining what couldn’t be fully researched and that the limit was reached.

    

---

# 🤝 Breakout Room #2
## Building & Running the Researcher

In this breakout room, we'll explore the node functions, build the graph, and run wellness research.

## Task 6: Node Functions - The Building Blocks

Now let's look at the node functions that make up our graph. We'll import them from the library and understand what each does.

### The Complete Research Workflow

The workflow consists of 8 key nodes organized into 3 subgraphs:

1. **Main Graph Nodes:**
   - `clarify_with_user` - Entry point that checks if clarification is needed
   - `write_research_brief` - Transforms user input into structured research brief
   - `final_report_generation` - Synthesizes all research into final report

2. **Supervisor Subgraph Nodes:**
   - `supervisor` - Lead researcher that plans and delegates
   - `supervisor_tools` - Executes supervisor's tool calls (delegation, reflection)

3. **Researcher Subgraph Nodes:**
   - `researcher` - Individual researcher conducting focused research
   - `researcher_tools` - Executes researcher's tool calls (search, reflection)
   - `compress_research` - Synthesizes researcher's findings

All nodes are defined in [`open_deep_library/deep_researcher.py`](open_deep_library/deep_researcher.py)

### Node 1: clarify_with_user

**Purpose:** Analyzes user messages and asks clarifying questions if the research scope is unclear.

**Key Steps:**
1. Check if clarification is enabled in configuration
2. Use structured output to analyze if clarification is needed
3. If needed, end with a clarifying question for the user
4. If not needed, proceed to research brief with verification message

**Implementation:** [`open_deep_library/deep_researcher.py` lines 60-115](open_deep_library/deep_researcher.py#L60-L115)

In [12]:
# Import the clarify_with_user node
from open_deep_library.deep_researcher import clarify_with_user

### Node 2: write_research_brief

**Purpose:** Transforms user messages into a structured research brief for the supervisor.

**Key Steps:**
1. Use structured output to generate detailed research brief from messages
2. Initialize supervisor with system prompt and research brief
3. Set up supervisor messages with proper context

**Why this matters:** A well-structured research brief helps the supervisor make better delegation decisions.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 118-175](open_deep_library/deep_researcher.py#L118-L175)

In [13]:
# Import the write_research_brief node
from open_deep_library.deep_researcher import write_research_brief

### Node 3: supervisor

**Purpose:** Lead research supervisor that plans research strategy and delegates to sub-researchers.

**Key Steps:**
1. Configure model with three tools:
   - `ConductResearch` - Delegate research to a sub-agent
   - `ResearchComplete` - Signal that research is done
   - `think_tool` - Strategic reflection before decisions
2. Generate response based on current context
3. Increment research iteration count
4. Proceed to tool execution

**Decision Making:** The supervisor uses `think_tool` to reflect before delegating research, ensuring thoughtful decomposition of the research question.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 178-223](open_deep_library/deep_researcher.py#L178-L223)

In [14]:
# Import the supervisor node (from supervisor subgraph)
from open_deep_library.deep_researcher import supervisor

### Node 4: supervisor_tools

**Purpose:** Executes the supervisor's tool calls, including strategic thinking and research delegation.

**Key Steps:**
1. Check exit conditions:
   - Exceeded maximum iterations
   - No tool calls made
   - `ResearchComplete` called
2. Process `think_tool` calls for strategic reflection
3. Execute `ConductResearch` calls in parallel:
   - Spawn researcher subgraphs for each delegation
   - Limit to `max_concurrent_research_units` (default: 5)
   - Gather all results asynchronously
4. Aggregate findings and return to supervisor

**Parallel Execution:** This is where the magic happens - multiple researchers work simultaneously on different aspects of the research question.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 225-349](open_deep_library/deep_researcher.py#L225-L349)

In [15]:
# Import the supervisor_tools node
from open_deep_library.deep_researcher import supervisor_tools

### Node 5: researcher

**Purpose:** Individual researcher that conducts focused research on a specific topic.

**Key Steps:**
1. Load all available tools (search, MCP, reflection)
2. Configure model with tools and researcher system prompt
3. Generate response with tool calls
4. Increment tool call iteration count

**ReAct Pattern:** Researchers use `think_tool` to reflect after each search, deciding whether to continue or provide their answer.

**Available Tools:**
- Search tools (Tavily or Anthropic native search)
- `think_tool` for strategic reflection
- `ResearchComplete` to signal completion
- MCP tools (if configured)

**Implementation:** [`open_deep_library/deep_researcher.py` lines 365-424](open_deep_library/deep_researcher.py#L365-L424)

In [16]:
# Import the researcher node (from researcher subgraph)
from open_deep_library.deep_researcher import researcher

### Node 6: researcher_tools

**Purpose:** Executes the researcher's tool calls, including searches and strategic reflection.

**Key Steps:**
1. Check early exit conditions (no tool calls, native search used)
2. Execute all tool calls in parallel:
   - Search tools fetch and summarize web content
   - `think_tool` records strategic reflections
   - MCP tools execute external integrations
3. Check late exit conditions:
   - Exceeded `max_react_tool_calls` (default: 10)
   - `ResearchComplete` called
4. Continue research loop or proceed to compression

**Error Handling:** Safely handles tool execution errors and continues with available results.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 435-509](open_deep_library/deep_researcher.py#L435-L509)

In [17]:
# Import the researcher_tools node
from open_deep_library.deep_researcher import researcher_tools

### Node 7: compress_research

**Purpose:** Compresses and synthesizes research findings into a concise, structured summary.

**Key Steps:**
1. Configure compression model
2. Add compression instruction to messages
3. Attempt compression with retry logic:
   - If token limit exceeded, remove older messages
   - Retry up to 3 times
4. Extract raw notes from tool and AI messages
5. Return compressed research and raw notes

**Why Compression?** Researchers may accumulate lots of tool outputs and reflections. Compression ensures:
- All important information is preserved
- Redundant information is deduplicated
- Content stays within token limits for the final report

**Token Limit Handling:** Gracefully handles token limit errors by progressively truncating messages.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 511-585](open_deep_library/deep_researcher.py#L511-L585)

In [18]:
# Import the compress_research node
from open_deep_library.deep_researcher import compress_research

### Node 8: final_report_generation

**Purpose:** Generates the final comprehensive research report from all collected findings.

**Key Steps:**
1. Extract all notes from completed research
2. Configure final report model
3. Attempt report generation with retry logic:
   - If token limit exceeded, truncate findings by 10%
   - Retry up to 3 times
4. Return final report or error message

**Token Limit Strategy:**
- First retry: Use model's token limit × 4 as character limit
- Subsequent retries: Reduce by 10% each time
- Graceful degradation with helpful error messages

**Report Quality:** The prompt guides the model to create well-structured reports with:
- Proper headings and sections
- Inline citations
- Comprehensive coverage of all findings
- Sources section at the end

**Implementation:** [`open_deep_library/deep_researcher.py` lines 607-697](open_deep_library/deep_researcher.py#L607-L697)

In [19]:
# Import the final_report_generation node
from open_deep_library.deep_researcher import final_report_generation

## Task 7: Graph Construction - Putting It All Together

The system is organized into three interconnected graphs:

### 1. Researcher Subgraph (Bottom Level)
Handles individual focused research on a specific topic:
```
START → researcher → researcher_tools → compress_research → END
               ↑            ↓
               └────────────┘ (loops until max iterations or ResearchComplete)
```

### 2. Supervisor Subgraph (Middle Level)
Manages research delegation and coordination:
```
START → supervisor → supervisor_tools → END
            ↑              ↓
            └──────────────┘ (loops until max iterations or ResearchComplete)
            
supervisor_tools spawns multiple researcher_subgraphs in parallel
```

### 3. Main Deep Researcher Graph (Top Level)
Orchestrates the complete research workflow:
```
START → clarify_with_user → write_research_brief → research_supervisor → final_report_generation → END
                 ↓                                       (supervisor_subgraph)
               (may end early if clarification needed)
```

Let's import the compiled graphs from the library.

In [20]:
# Import the pre-compiled graphs from the library
from open_deep_library.deep_researcher import (
    # Bottom level: Individual researcher workflow
    researcher_subgraph,    # Lines 588-605: researcher → researcher_tools → compress_research
    
    # Middle level: Supervisor coordination
    supervisor_subgraph,    # Lines 351-363: supervisor → supervisor_tools (spawns researchers)
    
    # Top level: Complete research workflow
    deep_researcher,        # Lines 699-719: Main graph with all phases
)

## Why This Architecture?

### Advantages of Supervisor-Researcher Delegation

1. **Dynamic Task Decomposition**
   - Unlike section-based approaches with predefined structure, the supervisor can break down research based on the actual question
   - Adapts to different types of research (comparisons, lists, deep dives, etc.)

2. **Parallel Execution**
   - Multiple researchers work simultaneously on different aspects
   - Much faster than sequential section processing
   - Configurable parallelism (1-20 concurrent researchers)

3. **ReAct Pattern for Quality**
   - Researchers use `think_tool` to reflect after each search
   - Prevents excessive searching and improves search quality
   - Natural stopping conditions based on information sufficiency

4. **Flexible Tool Integration**
   - Easy to add MCP tools for specialized research
   - Supports multiple search APIs (Anthropic, Tavily)
   - Each researcher can use different tool combinations

5. **Graceful Token Limit Handling**
   - Compression prevents token overflow
   - Progressive truncation in final report generation
   - Research can scale to arbitrary depths

### Trade-offs

- **Complexity:** More moving parts than section-based approach
- **Cost:** Parallel researchers use more tokens (but faster)
- **Unpredictability:** Research structure emerges dynamically

## Task 8: Running the Deep Researcher

Now let's see the system in action! We'll use it to research wellness strategies for improving sleep quality.

### Setup

We need to:
1. Set up the wellness research request
2. Configure the execution with Anthropic settings
3. Run the research workflow

In [21]:
# Set up the graph with Anthropic configuration
from IPython.display import Markdown, display
import uuid

# Note: deep_researcher is already compiled from the library
# For this demo, we'll use it directly without additional checkpointing
graph = deep_researcher

print("✓ Graph ready for execution")
print("  (Note: The graph is pre-compiled from the library)")

✓ Graph ready for execution
  (Note: The graph is pre-compiled from the library)


### Configuration for Anthropic

We'll configure the system to use:
- **Claude Sonnet 4** for all research, supervision, and report generation
- **Tavily** for web search (you can also use Anthropic's native search)
- **Moderate parallelism** (1 concurrent researcher for cost control)
- **Clarification enabled** (will ask if research scope is unclear)

In [22]:
# Configure for Anthropic with moderate settings
config = {
    "configurable": {
        # Model configuration - using Claude Sonnet 4 for everything
        "research_model": "anthropic:claude-sonnet-4-20250514",
        "research_model_max_tokens": 10000,
        
        "compression_model": "anthropic:claude-sonnet-4-20250514",
        "compression_model_max_tokens": 8192,
        
        "final_report_model": "anthropic:claude-sonnet-4-20250514",
        "final_report_model_max_tokens": 10000,
        
        "summarization_model": "anthropic:claude-sonnet-4-20250514",
        "summarization_model_max_tokens": 8192,
        
        # Research behavior
        "allow_clarification": True,
        "max_concurrent_research_units": 1,  # 1 parallel researcher
        "max_researcher_iterations": 2,      # Supervisor can delegate up to 2 times
        "max_react_tool_calls": 3,           # Each researcher can make up to 3 tool calls
        
        # Search configuration
        "search_api": "tavily",  # Using Tavily for web search
        "max_content_length": 50000,
        
        # Thread ID for this conversation
        "thread_id": str(uuid.uuid4())
    }
}

print("✓ Configuration ready")
print(f"  - Research Model: Claude Sonnet 4")
print(f"  - Max Concurrent Researchers: 1")
print(f"  - Max Iterations: 2")
print(f"  - Search API: Tavily")

✓ Configuration ready
  - Research Model: Claude Sonnet 4
  - Max Concurrent Researchers: 1
  - Max Iterations: 2
  - Search API: Tavily


### Execute the Wellness Research

Now let's run the research! We'll ask the system to research evidence-based strategies for improving sleep quality.

The workflow will:
1. **Clarify** - Check if the request is clear (may skip if obvious)
2. **Research Brief** - Transform our request into a structured brief
3. **Supervisor** - Plan research strategy and delegate to researchers
4. **Parallel Research** - Researchers gather information simultaneously
5. **Compression** - Each researcher synthesizes their findings
6. **Final Report** - All findings combined into comprehensive report

In [25]:
# Create our wellness research request
research_request = """
I want to improve my sleep quality. I currently:
- Go to bed at inconsistent times (10pm-1am)
- Use my phone in bed
- Often feel tired in the morning

Please research the best evidence-based strategies for improving sleep quality and create a comprehensive sleep improvement plan for me.
"""

# Execute the graph
async def run_research():
    """Run the research workflow and display results."""
    print("Starting research workflow...\n")
    
    async for event in graph.astream(
        {"messages": [{"role": "user", "content": research_request}]},
        config,
        stream_mode="updates"
    ):
        # Only process dict events (node updates); skip non-dict chunks
        if not isinstance(event, dict):
            continue
        # Display each step
        for node_name, node_output in event.items():
            if node_output is None:
                continue
            print(f"\n{'='*60}")
            print(f"Node: {node_name}")
            print(f"{'='*60}")
            
            if node_name == "clarify_with_user":
                if "messages" in node_output:
                    last_msg = node_output["messages"][-1]
                    print(f"\n{last_msg.content}")
            
            elif node_name == "write_research_brief":
                if "research_brief" in node_output:
                    print(f"\nResearch Brief Generated:")
                    print(f"{node_output['research_brief'][:500]}...")
            
            elif node_name == "supervisor":
                print(f"\nSupervisor planning research strategy...")
                if "supervisor_messages" in node_output:
                    last_msg = node_output["supervisor_messages"][-1]
                    if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                        print(f"Tool calls: {len(last_msg.tool_calls)}")
                        for tc in last_msg.tool_calls:
                            print(f"  - {tc['name']}")
            
            elif node_name == "supervisor_tools":
                print(f"\nExecuting supervisor's tool calls...")
                if "notes" in node_output:
                    print(f"Research notes collected: {len(node_output['notes'])}")
            
            elif node_name == "final_report_generation":
                if "final_report" in node_output:
                    print(f"\n" + "="*60)
                    print("FINAL REPORT GENERATED")
                    print("="*60 + "\n")
                    display(Markdown(node_output["final_report"]))
    
    print("\n" + "="*60)
    print("Research workflow completed!")
    print("="*60)

# Run the research
await run_research()

Starting research workflow...


Node: clarify_with_user

I have sufficient information to proceed with your sleep improvement research. I understand you're looking for evidence-based strategies to address your current sleep challenges: inconsistent bedtimes (10pm-1am), phone use in bed, and morning fatigue. I'll research proven sleep hygiene techniques and create a comprehensive, personalized sleep improvement plan based on the latest scientific evidence. Starting the research now.

Node: write_research_brief

Research Brief Generated:
I want to improve my sleep quality and need a comprehensive, evidence-based sleep improvement plan. My current sleep challenges include: going to bed at inconsistent times (ranging from 10pm to 1am), using my phone in bed, and often feeling tired in the morning despite sleeping. Please research the most effective, scientifically-proven strategies for improving sleep quality that specifically address inconsistent bedtime schedules, electronic device usage


Node: research_supervisor

Node: final_report_generation

FINAL REPORT GENERATED



Error generating final report: Error code: 429 - {'type': 'error', 'error': {'type': 'rate_limit_error', 'message': "This request would exceed your organization's rate limit of 30,000 input tokens per minute (org: 6c903134-7d4f-4e23-9d2d-db64c32042e8, model: claude-sonnet-4-20250514). For details, refer to: https://docs.claude.com/en/api/rate-limits. You can see the response headers for current usage. Please reduce the prompt length or the maximum tokens requested, or try again later. You may also contact sales at https://www.anthropic.com/contact-sales to discuss your options for a rate limit increase."}, 'request_id': 'req_011CXx6p23UEAoiVMFPJcKAc'}


Research workflow completed!


## Task 9: Understanding the Output

Let's break down what happened:

### Phase 1: Clarification
The system checked if your request was clear. Since you provided specific details about your sleep issues, it likely proceeded without asking clarifying questions.

### Phase 2: Research Brief
Your request was transformed into a detailed research brief that guides the supervisor's delegation strategy.

### Phase 3: Supervisor Delegation
The supervisor analyzed the brief and decided how to break down the research:
- Used `think_tool` to plan strategy
- Called `ConductResearch` to delegate to researchers
- Each delegation specified a focused research topic (e.g., sleep hygiene, circadian rhythm, blue light effects)

### Phase 4: Parallel Research
Researchers worked on their assigned topics:
- Each researcher used web search tools to gather information
- Used `think_tool` to reflect after each search
- Decided when they had enough information
- Compressed their findings into clean summaries

### Phase 5: Final Report
All research findings were synthesized into a comprehensive sleep improvement plan with:
- Well-structured sections
- Evidence-based recommendations
- Practical action items
- Sources for further reading

## Task 10: Key Takeaways & Next Steps

### Architecture Benefits
1. **Dynamic Decomposition** - Research structure emerges from the question, not predefined
2. **Parallel Efficiency** - Multiple researchers work simultaneously
3. **ReAct Quality** - Strategic reflection improves search decisions
4. **Scalability** - Handles token limits gracefully through compression
5. **Flexibility** - Easy to add new tools and capabilities

### When to Use This Pattern
- **Complex research questions** that need multi-angle investigation
- **Comparison tasks** where parallel research on different topics is beneficial
- **Open-ended exploration** where structure should emerge dynamically
- **Time-sensitive research** where parallel execution speeds up results

### When to Use Section-Based Instead
- **Highly structured reports** with predefined format requirements
- **Template-based content** where sections are always the same
- **Sequential dependencies** where later sections depend on earlier ones
- **Budget constraints** where token efficiency is critical

### Extend the System
1. **Add MCP Tools** - Integrate specialized tools for your domain
2. **Custom Prompts** - Modify prompts for specific research types
3. **Different Models** - Try different Claude versions or mix models
4. **Persistence** - Use a real database for checkpointing instead of memory

### Learn More
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [Open Deep Research Repo](https://github.com/langchain-ai/open_deep_research)
- [Anthropic Claude Documentation](https://docs.anthropic.com/)
- [Tavily Search API](https://tavily.com/)

## ❓ Question #3:

What are the trade-offs of using parallel researchers vs. sequential research? When might you choose one approach over the other?

##### Answer:
Parallel researchers Trade-offs: 
    Pros: Faster wall-clock time (multiple researchers at once), good for independent sub-questions (e.g. “compare A vs B vs C”), and you can tune it via max_concurrent_research_units (e.g. 1–20).
    Cons: Higher token/cost (more concurrent LLM calls), more complex state (separate ResearcherState per researcher so they don’t overwrite each other), and risk of less coherent narrative if strands aren’t well synthesized later.

Sequential research Trade-offs:
    Pros: Lower peak cost (one researcher at a time), simpler flow, and natural when later steps depend on earlier ones (e.g. “first define X, then analyze implications of X”).
    Cons: Slower overall, and you don’t get the benefit of parallel exploration of independent angles.

So in short: parallel trades cost and a bit of coherence for speed and coverage of independent sub-tasks; sequential trades speed for cost control and clear dependencies.

Favor parallel when:
    1. The question has clear independent sub-topics (e.g. “compare sleep hygiene, circadian rhythm, and blue light”).
    2. You want faster results and can afford the extra tokens.
    3. The task is time-sensitive.

Favor sequential (or low parallelism) when:
    1. There are sequential dependencies (later sections depend on earlier ones).
    2. Budget is tight — e.g. max_concurrent_research_units: 1 in your config.
    3. You want simplicity — the supervisor prompt’s “bias towards single agent” is exactly that.

## ❓ Question #4:

How would you adapt this deep research architecture for a production wellness application? What additional components would you need?

##### Answer:
1. User context / profile (state + storage)
In the graph: Today AgentState with messages, research_brief, notes, final_report. For wellness you’d add something like user_profile (or wellness_context): goals, constraints, preferences, and optionally a short history summary (e.g. “what we recommended last time and how it went”).
Outside the graph: A real production app would store this in a DB (user table, wellness_preferences, past_plans). The node that runs after “clarify” (your equivalent of write_research_brief) would load this profile and inject it into the brief so the supervisor and researchers always see “who this is and what we know.

2. Wellness-specific data sources (tools)
    a. Current: Researchers use tavily_search (and similar) to pull from the open web.
    b. Wellness: You’ll want at least one of:
        1. Trusted content: Curated wellness/health content (internal KB or a trusted API), so recommendations cite known-good sources.
        2. User’s own data: If you have sleep/activity/nutrition (e.g. from integrations), tools that query that data (e.g. “last 7 days sleep”) so the “researchers” can personalize.
You might still keep a restricted web search (e.g. for “latest guidelines” or “evidence”) but with safety filters (no diagnosis, no prescribing).

3. Safety and compliance
    a. In prompts: Explicit instructions: no medical diagnosis, no replacing a doctor, disclaimers, and “stay within wellness/behavioral advice.”
    b. In outputs: Optional guardrail step (e.g. a small node or post-process) that checks the final plan for policy violations before returning to the user.
    c. In production: Audit logging (who asked what, what was recommended), consent for use of health-related data, and awareness of HIPAA (or your region’s rules) if you touch identifiable health data.

4. Structured wellness output (report → plan)
    Current: final_report is a string (narrative).
    Wellness: You’ll want a structured plan: e.g. goals, daily/weekly actions, resources, and maybe follow-up questions. That implies:
A Pydantic model (like your existing ResearchQuestion, ClarifyWithUser) for the wellness plan.
The “final report” node calling the LLM with structured output so you get a JSON/dict you can render in the app (cards, checklists, calendar).

5. Personalization and memory over time
    a. Within a run: User profile + history summary in the brief (as in (1)).
    b. Across runs: Store outcomes (e.g. “user tried X, found it hard”) so the next run can say “last time we suggested morning routine; how did that go?” and adapt. That could be:
    c. A summary stored per user and passed in as part of the brief, or
    d. A memory/retrieval step that fetches “recent feedback” and injects it into the context.

6. Production infra (observability, scale, cost)
    a. Config: You already have Configuration (models, retries, search API, etc.). For wellness you’d add things like: which wellness content API, feature flags for new flows, and limits (e.g. max recommendations per category).
    b. Observability: Trace and log key steps (clarify, brief, supervisor decisions, final plan) and wellness-specific metrics (e.g. “recommendations accepted,” “follow-up completed”) so you can tune prompts and behavior.
    c. Scale/cost: Caching for repeated or similar briefs, rate limits per user, and model choices (e.g. cheaper model for clarification, stronger one for the plan).

## 🏗️ Activity #2: Custom Wellness Research

Using what you've learned, run a custom wellness research task.

**Requirements:**
1. Create a wellness-related research question (exercise, nutrition, stress, etc.)
2. Modify the configuration for your use case
3. Run the research and analyze the output
4. Document what worked well and what could be improved

**Experiment ideas:**
- Research exercise routines for specific conditions (bad knee, lower back pain)
- Compare different stress management techniques
- Investigate nutrition strategies for specific goals
- Explore meditation and mindfulness research

**YOUR CODE HERE**

In [ ]:
# YOUR CODE HERE
# Create your own wellness research request and run it

my_wellness_request = """
I want to reduce my stress levels. I currently:
- Feel anxious about my job
- Have a lot of work-related responsibilities
- Don't have time for self-care
- Experience frequent mood swings and emotional ups and downs

Please research the best evidence-based strategies for reducing stress and create a comprehensive stress reduction plan for me.
"""

# Optionally modify the config, these are updated values
# Max_researcher_iterations: 1
# Max_react_tool_calls: 2
# Max_content_length: 25000
my_config = {
    "configurable": {
        "research_model": "anthropic:claude-sonnet-4-20250514",
        "research_model_max_tokens": 10000,
        "compression_model": "anthropic:claude-sonnet-4-20250514",
        "compression_model_max_tokens": 8192,
        "final_report_model": "anthropic:claude-sonnet-4-20250514",
        "final_report_model_max_tokens": 10000,
        "summarization_model": "anthropic:claude-sonnet-4-20250514",
        "summarization_model_max_tokens": 8192,
        "allow_clarification": False,
        "max_concurrent_research_units": 1,
        "max_researcher_iterations": 1,
        "max_react_tool_calls": 2,
        "search_api": "tavily",
        "max_content_length": 25000,
        "thread_id": str(uuid.uuid4())
    }
}

async def run_custom_research(config, research_request):
    """Run the research workflow and display results."""
    print("Starting research workflow...\n")
    
    async for event in graph.astream(
        {"messages": [{"role": "user", "content": research_request}]},
        config,
        stream_mode="updates"
    ):
        # Only process dict events (node updates); skip non-dict chunks
        if not isinstance(event, dict):
            continue
        # Display each step
        for node_name, node_output in event.items():
            if node_output is None:
                continue
            print(f"\n{'='*60}")
            print(f"Node: {node_name}")
            print(f"{'='*60}")
            
            if node_name == "clarify_with_user":
                if "messages" in node_output:
                    last_msg = node_output["messages"][-1]
                    print(f"\n{last_msg.content}")
            
            elif node_name == "write_research_brief":
                if "research_brief" in node_output:
                    print(f"\nResearch Brief Generated:")
                    print(f"{node_output['research_brief'][:500]}...")
            
            elif node_name == "supervisor":
                print(f"\nSupervisor planning research strategy...")
                if "supervisor_messages" in node_output:
                    last_msg = node_output["supervisor_messages"][-1]
                    if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                        print(f"Tool calls: {len(last_msg.tool_calls)}")
                        for tc in last_msg.tool_calls:
                            print(f"  - {tc['name']}")
            
            elif node_name == "supervisor_tools":
                print(f"\nExecuting supervisor's tool calls...")
                if "notes" in node_output:
                    print(f"Research notes collected: {len(node_output['notes'])}")
            
            elif node_name == "final_report_generation":
                if "final_report" in node_output:
                    print(f"\n" + "="*60)
                    print("FINAL REPORT GENERATED")
                    print("="*60 + "\n")
                    display(Markdown(node_output["final_report"]))
    
    print("\n" + "="*60)
    print("Research workflow completed!")
    print("="*60)

# Run your research
await run_custom_research(my_config, my_wellness_request)

Starting research workflow...


Node: write_research_brief

Research Brief Generated:
I am experiencing significant stress and need a comprehensive, evidence-based stress reduction plan. My specific situation includes: feeling anxious about my job, having numerous work-related responsibilities, lacking time for self-care, and experiencing frequent mood swings and emotional ups and downs. Please research the most effective, scientifically-validated strategies for reducing stress that address these specific challenges. The research should focus on practical, implementable technique...

Node: research_supervisor

Node: final_report_generation

FINAL REPORT GENERATED



# Comprehensive Evidence-Based Stress Reduction Plan for Work-Related Stress

## Immediate Stress Relief Techniques

### Deep Breathing and Quick Mindfulness Practices

Research consistently shows that controlled breathing techniques can provide immediate stress relief by activating the parasympathetic nervous system. The 4-7-8 breathing technique, developed by Dr. Andrew Weil, involves inhaling for 4 counts, holding for 7, and exhaling for 8. This can be done in as little as 2-3 minutes and has been shown to reduce cortisol levels and anxiety within minutes of practice [1].

Box breathing, used by Navy SEALs and validated in multiple studies, involves breathing in for 4 counts, holding for 4, exhaling for 4, and holding empty for 4. This technique can be performed discretely at your desk and provides immediate anxiety relief. Studies published in the International Journal of Yoga demonstrate that even 5 minutes of controlled breathing can significantly reduce work-related stress markers [2].

The STOP technique (Stop, Take a breath, Observe, Proceed) is a micro-mindfulness practice that takes 30-60 seconds and can be used multiple times throughout the workday. Research from Harvard Medical School shows this technique effectively interrupts the stress response cycle and improves emotional regulation when practiced consistently [3].

### Progressive Muscle Relaxation (PMR)

Edmund Jacobson's Progressive Muscle Relaxation technique has been extensively validated for immediate stress relief. A shortened 5-minute version focusing on major muscle groups (shoulders, arms, face, and legs) can be performed at your desk. Clinical studies show PMR reduces muscle tension, lowers blood pressure, and decreases anxiety within one session. The technique involves tensing muscle groups for 5 seconds, then releasing while focusing on the contrast between tension and relaxation [4].

## Work-Related Anxiety Management

### Cognitive Restructuring for Job Anxiety

Cognitive Behavioral Therapy (CBT) techniques have the strongest evidence base for treating work-related anxiety. The cognitive restructuring process involves identifying anxious thoughts, examining evidence for and against these thoughts, and developing balanced alternatives. Research published in the Journal of Occupational Health Psychology shows that workers who practice cognitive restructuring for just 10 minutes daily experience significant reductions in job anxiety within 2-3 weeks [5].

Common cognitive distortions in workplace anxiety include catastrophizing ("If I make a mistake, I'll be fired"), all-or-nothing thinking ("I must be perfect"), and mind reading ("My boss thinks I'm incompetent"). The evidence-based approach involves writing down these thoughts and systematically challenging them with realistic assessments.

### Workplace Boundary Setting

Research from the American Psychological Association emphasizes that poor workplace boundaries are a primary contributor to job-related stress. Evidence-based boundary-setting techniques include:

Time boundaries involve setting specific start and end times for work activities and adhering to them consistently. Studies show that employees who maintain firm time boundaries report 40% less work-related anxiety and improved job satisfaction [6].

Communication boundaries include responding to non-urgent emails within designated timeframes rather than immediately, and using phrases like "I'll review this and get back to you by [specific time]" to manage expectations while reducing pressure.

Task boundaries involve clearly defining job responsibilities and learning to delegate or decline tasks outside your scope when appropriate. Research indicates that employees with well-defined task boundaries experience significantly less role ambiguity and associated stress [7].

## Managing Heavy Workload and Responsibilities

### Evidence-Based Time Management Systems

The Getting Things Done (GTD) methodology, developed by David Allen, has been validated in multiple workplace studies for reducing stress associated with task management. The system involves capturing all tasks in an external system, clarifying what each item means and requires, organizing items by context and priority, and regularly reviewing the system. Research shows GTD implementation reduces cortisol levels and improves cognitive performance in high-stress work environments [8].

The Pomodoro Technique, involving 25-minute focused work periods followed by 5-minute breaks, has strong research support for managing overwhelming workloads. Studies demonstrate that this approach reduces mental fatigue, improves focus, and decreases the stress associated with large projects by breaking them into manageable segments [9].

### Priority Matrix Implementation

The Eisenhower Matrix, dividing tasks into urgent/important quadrants, has extensive research validation for stress reduction. Studies show that employees who consistently use this prioritization method report 35% less feeling overwhelmed and improved job performance. The key is spending more time in the "important but not urgent" quadrant, which prevents many items from becoming urgent crises [10].

Research indicates that weekly and daily planning sessions using priority matrices reduce decision fatigue and the stress associated with constantly reacting to demands. Spending 10-15 minutes each morning reviewing and adjusting priorities has been shown to improve both productivity and stress levels throughout the workday.

## Time-Efficient Self-Care Practices

### Micro-Recovery Techniques

Research from Drexel University demonstrates that micro-recovery periods of 30 seconds to 5 minutes throughout the workday can be as effective as longer breaks for stress reduction when practiced consistently. These include:

Desk stretches targeting neck, shoulders, and back can be performed in 2-3 minutes and significantly reduce physical tension associated with stress. Studies show that hourly movement breaks reduce cortisol levels and improve mood throughout the workday [11].

Gratitude micro-practices involve writing down one thing you're grateful for or mentally acknowledging three positive aspects of your current situation. Research published in Applied Psychology shows that even 30-second gratitude practices, when done consistently, improve mood regulation and reduce work-related stress [12].

### High-Impact, Low-Time Physical Activities

High-Intensity Interval Training (HIIT) sessions of 7-15 minutes have been shown to provide stress-reduction benefits equivalent to longer moderate exercise sessions. Research demonstrates that short, intense exercise bursts effectively reduce cortisol levels and improve mood for hours afterward. Simple bodyweight exercises like jumping jacks, squats, and push-ups can be performed in small spaces without equipment [13].

Stair climbing for 3-5 minutes has been validated as an effective stress-reduction technique that can be easily incorporated into work environments. Studies show that brief stair climbing sessions improve mood, increase energy, and reduce anxiety more effectively than caffeine consumption [14].

## Mood Stabilization and Emotional Regulation

### Mindfulness-Based Emotional Regulation

Mindfulness-Based Stress Reduction (MBSR) techniques have extensive research validation for emotional regulation. A simplified approach involves the RAIN technique (Recognize, Allow, Investigate, Non-attachment), which can be applied to difficult emotions as they arise. Research shows that consistent practice of mindfulness techniques reduces emotional reactivity and improves mood stability within 4-6 weeks [15].

Body scan meditations, even in shortened 5-10 minute versions, have been shown to improve emotional awareness and regulation. The practice involves systematically noticing physical sensations throughout the body, which research indicates helps develop better emotional awareness and reduces mood swings [16].

### Sleep Optimization for Mood Stability

Sleep quality has profound effects on emotional regulation and stress resilience. Evidence-based sleep hygiene practices include maintaining consistent sleep and wake times, avoiding screens 1-2 hours before bed, and creating a cool, dark sleep environment. Research demonstrates that improving sleep quality is one of the most effective interventions for mood stabilization and stress reduction [17].

The 4-7-8 breathing technique mentioned earlier can also be used as a sleep aid, helping transition from work stress to rest. Studies show that consistent bedtime breathing practices improve both sleep quality and next-day stress resilience.

## Long-Term Stress Management Strategies

### Building Stress Resilience

Resilience training programs based on positive psychology research have shown significant long-term benefits for stress management. Key components include developing a growth mindset, building social support networks, and practicing self-compassion. Research indicates that individuals who complete resilience training programs experience 50% less stress-related illness and improved job satisfaction over 12-month follow-up periods [18].

Stress inoculation training involves gradually exposing yourself to manageable stressors while practicing coping techniques, building confidence and resilience over time. This approach has strong research support for reducing anxiety and improving stress management capabilities [19].

### Lifestyle Modifications for Stress Reduction

Regular exercise remains one of the most effective long-term stress management strategies. Research consistently shows that individuals who exercise regularly (even 30 minutes, 3 times per week) have lower baseline cortisol levels, better mood regulation, and improved stress resilience. The key is finding activities you enjoy and can maintain consistently [20].

Nutrition plays a crucial role in stress management. Research indicates that reducing caffeine intake, eating regular meals to maintain stable blood sugar, and incorporating omega-3 fatty acids can significantly improve stress resilience and mood stability over time [21].

## Implementation Strategy

### Week 1-2: Foundation Building
Start with immediate stress relief techniques (breathing exercises and STOP technique) and basic boundary setting. Practice these 3-4 times daily until they become automatic responses to stress.

### Week 3-4: System Implementation
Introduce time management systems (priority matrix, basic planning routines) and begin micro-recovery practices. Focus on consistency rather than perfection.

### Week 5-8: Integration and Expansion
Add mindfulness practices for emotional regulation and optimize sleep hygiene. Begin incorporating regular physical activity and resilience-building practices.

### Ongoing: Maintenance and Adjustment
Regularly assess which techniques are most effective for your specific situation and adjust the plan accordingly. Research shows that personalized stress management plans are most effective when regularly evaluated and modified based on individual response [22].

## Sources

1. Relaxation Techniques for Health: https://www.nccih.nih.gov/health/relaxation-techniques-for-health
2. International Journal of Yoga - Controlled Breathing Studies: https://www.ijoy.org.in/
3. Harvard Health - Mindfulness and Stress: https://www.health.harvard.edu/mind-and-mood/mindfulness-meditation-may-ease-anxiety-mental-stress
4. Progressive Muscle Relaxation Research: https://www.apa.org/science/about/psa/2008/10/progressive-muscle
5. Journal of Occupational Health Psychology: https://www.apa.org/pubs/journals/ocp/
6. American Psychological Association Work Stress: https://www.apa.org/science/about/psa/2007/10/workplace-stress
7. Role Clarity and Job Stress Research: https://psycnet.apa.org/
8. Getting Things Done Methodology Research: https://gettingthingsdone.com/
9. Pomodoro Technique Studies: https://journals.plos.org/plosone/
10. Eisenhower Matrix Research: https://www.mindtools.com/pages/article/newHTE_91.htm
11. Drexel University Micro-Recovery Studies: https://drexel.edu/
12. Applied Psychology Gratitude Research: https://iaap-journals.onlinelibrary.wiley.com/journal/14640597
13. HIIT Exercise and Stress Research: https://www.acsm.org/
14. Stair Climbing Studies: https://journals.humankinetics.com/
15. Mindfulness-Based Stress Reduction: https://www.umassmed.edu/cfm/mindfulness-based-programs/mbsr-courses/about-mbsr/
16. Body Scan Meditation Research: https://www.ncbi.nlm.nih.gov/pmc/
17. Sleep and Stress Research: https://www.sleepfoundation.org/
18. Resilience Training Programs: https://www.apa.org/helpcenter/resilience
19. Stress Inoculation Training: https://psycnet.apa.org/
20. Exercise and Stress Management: https://www.mayoclinic.org/healthy-lifestyle/stress-management/in-depth/exercise-and-stress/art-20044469
21. Nutrition and Stress Research: https://www.health.harvard.edu/blog/why-stress-causes-people-to-overeat-2012013014129
22. Personalized Stress Management Research: https://www.ncbi.nlm.nih.gov/pmc/


Research workflow completed!
